# Step 2: Loading data to gluon and obtain features of images
** Extract features based on [Gluon Model Zoo](https://mxnet.incubator.apache.org/versions/master/api/python/gluon/model_zoo.html)**

In [1]:
import mxnet as mx
from mxnet import autograd
from mxnet import gluon
from mxnet import image
from mxnet import init
from mxnet import nd
from mxnet.gluon.data import vision
from mxnet.gluon.model_zoo import vision as models
import numpy as np
from tqdm import tqdm

import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import warnings
warnings.filterwarnings("ignore")

E:\Anaconda\envs\MXNet\lib\site-packages\mxnet-1.0.1-py2.7.egg\mxnet\optimizer.py:136: UserWarning: WARNING: New optimizer mxnet.optimizer.NAG is overriding existing optimizer mxnet.optimizer.NAG
  Optimizer.opt_registry[name].__name__))


**[Image API in MXNet](https://mxnet.incubator.apache.org/api/python/image/image.html)**

**[Gluon Data API](https://mxnet.incubator.apache.org/api/python/gluon/data.html?highlight=imagefolderdataset#mxnet.gluon.data.vision.ImageFolderDataset)**

**[Fine-tuning: 通过微调来迁移学习](http://zh.gluon.ai/chapter_computer-vision/fine-tuning.html)**

# 2.1 Define the pre-processing function

**这里需要mean和std同时都设置正确才能进行预处理，如果你只设置了mean，没有设置std，那么还是没有启动归一化的预处理。
 这里主要调用ColorNormalizeAug()函数，这个函数调用color_normalize()函数，这个函数的实现很简单，
 就是将原图像的像素值减去均值mean，然后除以标准差std得到返回值。**
![image](https://raw.githubusercontent.com/Trouble404/Kaggle-Dog-breed-Identification/master/readme_pic_add/colornormalizeaug.PNG)

**resize这个参数很重要，一般都要做resize，如果你的resize参数设置为224，你的原图像是350X300，那么最后resize的大小就是
 (350X300/224)X224。这里ResizeAug()函数调用resize_short()函数，resize_short()函数调用OpenCV的imresize()函数完成resize
 ，interp参数为2表示采用双三次插值做resize。**
![iamge](https://raw.githubusercontent.com/Trouble404/Kaggle-Dog-breed-Identification/master/readme_pic_add/forceresizeaug.PNG)

**Pretrained models are converted from torchvision. All pre-trained models expect input images normalized in the same way, i.e. mini-batches of 3-channel RGB images of shape (N x 3 x H x W), where N is the batch size, and H and W are expected to be at least 224. The images have to be loaded in to a range of [0, 1] and then normalized using ```mean = [0.485, 0.456, 0.406]``` and ```std = [0.229, 0.224, 0.225]```. The transformation should preferrably happen at preprocessing. **
 

In [2]:
ctx = mx.cpu() # use cpu   gpu will cause python stop working here

preprocessing = [
    image.ForceResizeAug((224,224)), # Make resize shorter edge to size augmenter.
    image.ColorNormalizeAug(mean=nd.array([0.485, 0.456, 0.406]), std=nd.array([0.229, 0.224, 0.225])) # Mean and std normalization
]

def transform(data, label):
    data = data.astype('float32') / 255  # transform type to float and do normalization
    for pre in preprocessing:
        data = pre(data) # preprocessing data
    
    data = nd.transpose(data, (2,0,1)) # Transpose image to N, H, W  Kaggle image is BGR but we need RGB here
    return data, nd.array([label]).asscalar().astype('float32') # return to ndARRARY

# 2.2 define the output feature
![image](https://raw.githubusercontent.com/Trouble404/Kaggle-Dog-breed-Identification/master/readme_pic_add/asincontent.PNG)

In [3]:
def get_features(net, data):
    features = []
    labels = []

    for X, y in tqdm(data): # tqdm to display the schedule
        feature = net.features(X.as_in_context(ctx)) # return target arrary of image features
        features.append(feature.asnumpy()) # asnumpy() -> convert to numpy array  superposition features
        labels.append(y.asnumpy()) # superposition labels
    
    features = np.concatenate(features, axis=0) # splice big data fast
    labels = np.concatenate(labels, axis=0)
    return features, labels

# 2.3 obtain feature vector

**inception_v3 model need resize at least 299, other models need resize at least 244**
![iamge](https://raw.githubusercontent.com/Trouble404/Kaggle-Dog-breed-Identification/master/readme_pic_add/loading.png)

![iamge](https://raw.githubusercontent.com/Trouble404/Kaggle-Dog-breed-Identification/master/readme_pic_add/dataloder.PNG)

## for train data

In [4]:
preprocessing[0] = image.ForceResizeAug((224,224))
imgs = vision.ImageFolderDataset('for_train', transform=transform) # read sorted images
data = gluon.data.DataLoader(imgs, 64) # load images

**VGG-16 model with batch normalization from the [Very Deep Convolutional Networks for Large-Scale Image Recognition](https://arxiv.org/abs/1409.1556) paper.**

**ResNet-152 V1 model from [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385) paper.**

**Densenet-BC 161-layer model from the [Densely Connected Convolutional Networks](https://arxiv.org/pdf/1608.06993.pdf) paper.**

In [5]:
%%time
features_vgg, _ = get_features(models.vgg16_bn(pretrained=True, ctx=ctx), data)
features_resnet, _ = get_features(models.resnet152_v1(pretrained=True, ctx=ctx), data)
features_densenet, _ = get_features(models.densenet161(pretrained=True, ctx=ctx), data)

100%|██████████████████████████████████████████████████████████████████████████████| 160/160 [1:29:11<00:00, 33.45s/it]


Wall time: 5h 25min 4s


In [9]:
features_vgg, labels = get_features(models.vgg16_bn(pretrained=True, ctx=ctx), data)

100%|██████████████████████████████████████████████████████████████████████████████| 160/160 [2:17:23<00:00, 51.52s/it]


**Inception v3 model from [Rethinking the Inception Architecture for Computer Vision](https://arxiv.org/abs/1512.00567) paper.**

In [11]:
preprocessing[0] = image.ForceResizeAug((299,299))
imgs_299 = vision.ImageFolderDataset('for_train', transform=transform)
data_299 = gluon.data.DataLoader(imgs_299, 64)

In [12]:
features_inception, _ = get_features(models.inception_v3(pretrained=True, ctx=ctx), data)

Model file is not found. Downloading.


100%|██████████████████████████████████████████████████████████████████████████████| 160/160 [1:07:40<00:00, 25.38s/it]


In [6]:
import h5py  # compress and save features

In [13]:
with h5py.File('features_train.h5', 'w') as f:
    f['vgg'] = features_vgg
    f['resnet'] = features_resnet
    f['densenet'] = features_densenet
    f['inception'] = features_inception
    f['labels'] = labels

## for test data

In [15]:
preprocessing[0] = image.ForceResizeAug((224,224))
imgs = vision.ImageFolderDataset('for_test', transform=transform)
data = gluon.data.DataLoader(imgs, 64)

features_vgg, _ = get_features(models.vgg16_bn(pretrained=True, ctx=ctx), data)
features_resnet, _ = get_features(models.resnet152_v1(pretrained=True, ctx=ctx), data)
features_densenet, _ = get_features(models.densenet161(pretrained=True, ctx=ctx), data)


100%|██████████████████████████████████████████████████████████████████████████████| 162/162 [2:19:34<00:00, 51.70s/it]

100%|██████████████████████████████████████████████████████████████████████████████| 162/162 [1:54:31<00:00, 42.41s/it]

100%|██████████████████████████████████████████████████████████████████████████████| 162/162 [1:36:34<00:00, 35.77s/it]


In [16]:
preprocessing[0] = image.ForceResizeAug((299,299))
imgs_299 = vision.ImageFolderDataset('for_test', transform=transform)
data_299 = gluon.data.DataLoader(imgs_299, 64)

In [17]:
features_inception, _ = get_features(models.inception_v3(pretrained=True, ctx=ctx), data)


100%|██████████████████████████████████████████████████████████████████████████████| 162/162 [1:02:40<00:00, 23.21s/it]


In [18]:
with h5py.File('features_test.h5', 'w') as f:
    f['vgg'] = features_vgg
    f['resnet'] = features_resnet
    f['densenet'] = features_densenet
    f['inception'] = features_inception